<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="../../resources/logo.png" alt="Intellinum Bootcamp" style="width: 400px; height: 200px">
</div>

# Querying JSON & Hierarchical Data with DataFrames

Apache Spark&trade; make it easy to work with hierarchical data, such as nested JSON records.

## In this lesson you:
* Use DataFrames to query JSON data.
* Query nested structured data.
* Query data containing array columns.


In [1]:
#MODE = "LOCAL"
MODE = "CLUSTER"

import sys
from pyspark.sql import SparkSession
from pyspark import SparkConf
import os
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark import SparkConf
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.storagelevel import StorageLevel
from matplotlib import interactive
interactive(True)
import matplotlib.pyplot as plt
%matplotlib inline
import json
import math
import numbers
import numpy as np
import plotly
plotly.offline.init_notebook_mode(connected=True)

sys.path.insert(0,'../../src')
from settings import *

try:
    fh = open('../../libs/pyspark24_py36.zip', 'r')
except FileNotFoundError:
    !AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY} AWS_SECRET_ACCESS_KEY={AWS_SECRET_KEY} aws s3 cp s3://yuan.intellinum.co/bins/pyspark24_py36.zip ../../libs/pyspark24_py36.zip

try:
    spark.stop()
    print("Stopped a SparkSession")
except Exception as e:
    print("No existing SparkSession detected")
    print("Creating a new SparkSession")

SPARK_DRIVER_MEMORY= "1G"
SPARK_DRIVER_CORE = "1"
SPARK_EXECUTOR_MEMORY= "1G"
SPARK_EXECUTOR_CORE = "1"
SPARK_EXECUTOR_INSTANCES = 6



conf = None
if MODE == "LOCAL":
    os.environ["PYSPARK_PYTHON"] = "/home/yuan/anaconda3/envs/pyspark24_py36/bin/python"
    conf = SparkConf().\
            setAppName("pyspark_day03_querying_json").\
            setMaster('local[*]').\
            set('spark.driver.maxResultSize', '0').\
            set('spark.jars', '../../libs/mysql-connector-java-5.1.45-bin.jar').\
            set('spark.jars.packages','net.java.dev.jets3t:jets3t:0.9.0,com.google.guava:guava:16.0.1,com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-aws:2.7.1')
else:
    os.environ["PYSPARK_PYTHON"] = "./MN/pyspark24_py36/bin/python"
    conf = SparkConf().\
            setAppName("pyspark_day03_querying_json").\
            setMaster('yarn-client').\
            set('spark.executor.cores', SPARK_EXECUTOR_CORE).\
            set('spark.executor.memory', SPARK_EXECUTOR_MEMORY).\
            set('spark.driver.cores', SPARK_DRIVER_CORE).\
            set('spark.driver.memory', SPARK_DRIVER_MEMORY).\
            set("spark.executor.instances", SPARK_EXECUTOR_INSTANCES).\
            set('spark.sql.files.ignoreCorruptFiles', 'true').\
            set('spark.yarn.dist.archives', '../../libs/pyspark24_py36.zip#MN').\
            set('spark.sql.shuffle.partitions', '5000').\
            set('spark.default.parallelism', '5000').\
            set('spark.driver.maxResultSize', '0').\
            set('spark.jars.packages','net.java.dev.jets3t:jets3t:0.9.0,com.google.guava:guava:16.0.1,com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-aws:2.7.1'). \
            set('spark.driver.maxResultSize', '0').\
            set('spark.jars', 's3://yuan.intellinum.co/bins/mysql-connector-java-5.1.45-bin.jar')
        

spark = SparkSession.builder.\
    config(conf=conf).\
    getOrCreate()


sc = spark.sparkContext

sc.addPyFile('../../src/settings.py')

sc=spark.sparkContext
hadoop_conf = sc._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("fs.s3a.access.key", AWS_ACCESS_KEY)
hadoop_conf.set("fs.s3a.secret.key", AWS_SECRET_KEY)
hadoop_conf.set("mapreduce.fileoutputcommitter.algorithm.version", "2")

def display(df, limit=10):
    return df.limit(limit).toPandas()

def dfTest(id, expected, result):
    assert str(expected) == str(result), "{} does not equal expected {}".format(result, expected)

No existing SparkSession detected
Creating a new SparkSession


## Examining the Contents of a JSON file

JSON is a common file format used in big data applications and in data lakes (or large stores of diverse data).  File formats such as JSON arise out of a number of data needs.  For instance, what if:
<br>
* Your schema, or the structure of your data, changes over time?
* You need nested fields like an array with many values or an array of arrays?
* You don't know how you're going use your data yet, so you don't want to spend time creating relational tables?

The popularity of JSON is largely due to the fact that JSON allows for nested, flexible schemas.

This lesson uses the `s3://data.intellinum.co/bootcamp/common/blog.json`. If you examine the raw file, notice it contains compact JSON data. There's a single JSON object on each line of the file; each object corresponds to a row in the table. Each row represents a blog post and the json file contains all blog posts through August 9, 2017.

In [2]:
!AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY} AWS_SECRET_ACCESS_KEY={AWS_SECRET_KEY} aws s3 cp s3://data.intellinum.co/bootcamp/common/blog.json - | head -n 2
    

{"status": "publish", "description": null, "creator": "roy", "link": "https://databricks.com/blog/2014/04/10/mapr-integrates-spark-stack.html", "authors": ["Tomer Shiran (VP of Product Management at MapR)"], "id": 33, "categories": ["Company Blog", "Partners"], "dates": {"publishedOn": "2014-04-10", "tz": "UTC", "createdOn": "2014-04-10"}, "title": "MapR Integrates the Complete Apache Spark Stack", "slug": "mapr-integrates-spark-stack", "content": "<div class=\"post-meta\">This post is guest authored by our friends at MapR, announcing our new partnership to provide enterprise support for Apache Spark as part of MapR's Distribution of Hadoop.</div>\n\n<hr />\n\nWith over 500 paying customers, my team and I have the opportunity to talk to many organizations that are leveraging Hadoop in production to extract value from big data. One of the most common topics raised by our customers in recent months is Apache Spark. Some customers just want to learn more about the advantages of this techn

Create a DataFrame out of the syntax introduced in the previous lesson:

In [3]:
blogDF = spark.read.option("inferSchema","true").option("header","true").json("s3a://data.intellinum.co/bootcamp/common/blog.json")

Take a look at the schema by invoking `printSchema` method.

In [4]:
blogDF.printSchema()

root
 |-- authors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- content: string (nullable = true)
 |-- creator: string (nullable = true)
 |-- dates: struct (nullable = true)
 |    |-- createdOn: string (nullable = true)
 |    |-- publishedOn: string (nullable = true)
 |    |-- tz: string (nullable = true)
 |-- description: string (nullable = true)
 |-- id: long (nullable = true)
 |-- link: string (nullable = true)
 |-- slug: string (nullable = true)
 |-- status: string (nullable = true)
 |-- title: string (nullable = true)



Run a query to view the contents of the table.

Notice:
* The `authors` column is an array containing one or more author names.
* The `categories` column is an array of one or more blog post category names.
* The `dates` column contains nested fields `createdOn`, `publishedOn` and `tz`.

In [5]:
display(blogDF.select("authors","categories","dates","content"))

,authors,categories,dates,content
0,[Tomer Shiran (VP of Product Management at MapR)],"[Company Blog, Partners]","(2014-04-10, 2014-04-10, UTC)","<div class=""post-meta"">This post is guest auth..."
1,[Tathagata Das],"[Apache Spark, Engineering Blog, Machine Learn...","(2014-04-10, 2014-04-10, UTC)",We are happy to announce the availability of <...
2,[Steven Hillion],"[Company Blog, Partners]","(2014-04-01, 2014-04-01, UTC)","<div class=""post-meta"">This post is guest auth..."
3,"[Michael Armbrust, Reynold Xin]","[Apache Spark, Engineering Blog]","(2014-03-27, 2014-03-27, UTC)",Building a unified platform for big data analy...
4,[Patrick Wendell],"[Apache Spark, Engineering Blog]","(2014-02-04, 2014-02-04, UTC)",Our goal with Apache Spark is very simple: pro...
5,"[Ali Ghodsi, Ahir Reddy]","[Apache Spark, Ecosystem, Engineering Blog]","(2014-01-02, 2014-01-02, UTC)",Apache Hadoop integration has always been a ke...
6,[Russell Cardullo (Data Infrastructure Enginee...,"[Company Blog, Customers]","(2014-03-26, 2014-03-26, UTC)","<div class=""post-meta"">We're very happy to see..."
7,"[Jai Ranganathan, Matei Zaharia]","[Apache Spark, Engineering Blog]","(2014-03-21, 2014-03-21, UTC)","<div class=""post-meta"">\n\nThis article was cr..."
8,[Databricks Press Office],"[Announcements, Company Blog]","(2014-03-19, 2014-03-19, UTC)","<strong>BERKELEY, Calif. – March 18, 2014 –</s..."
9,[Ion Stoica],"[Apache Spark, Engineering Blog]","(2014-03-03, 2014-03-03, UTC)","<div class=""blogContent"">\n\nWe are delighted ..."


## Nested Data

Think of nested data as columns within columns. 

For instance, look at the `dates` column.

In [6]:
datesDF = blogDF.select("dates")
display(datesDF)

,dates
0,"(2014-04-10, 2014-04-10, UTC)"
1,"(2014-04-10, 2014-04-10, UTC)"
2,"(2014-04-01, 2014-04-01, UTC)"
3,"(2014-03-27, 2014-03-27, UTC)"
4,"(2014-02-04, 2014-02-04, UTC)"
5,"(2014-01-02, 2014-01-02, UTC)"
6,"(2014-03-26, 2014-03-26, UTC)"
7,"(2014-03-21, 2014-03-21, UTC)"
8,"(2014-03-19, 2014-03-19, UTC)"
9,"(2014-03-03, 2014-03-03, UTC)"


Pull out a specific subfield with `.` (object) notation.

In [7]:
display(blogDF.select("dates.createdOn", "dates.publishedOn"))

,createdOn,publishedOn
0,2014-04-10,2014-04-10
1,2014-04-10,2014-04-10
2,2014-04-01,2014-04-01
3,2014-03-27,2014-03-27
4,2014-02-04,2014-02-04
5,2014-01-02,2014-01-02
6,2014-03-26,2014-03-26
7,2014-03-21,2014-03-21
8,2014-03-19,2014-03-19
9,2014-03-03,2014-03-03


Create a DataFrame, `blog2DF` that contains the original columns plus the new `publishedOn` column obtained
from flattening the dates column.

In [8]:
from pyspark.sql.functions import col
blog2DF = blogDF.withColumn("publishedOn",col("dates.publishedOn"))

With this temporary view, apply the printSchema method to check its schema and confirm the timestamp conversion.

In [9]:
blog2DF.printSchema()

root
 |-- authors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- content: string (nullable = true)
 |-- creator: string (nullable = true)
 |-- dates: struct (nullable = true)
 |    |-- createdOn: string (nullable = true)
 |    |-- publishedOn: string (nullable = true)
 |    |-- tz: string (nullable = true)
 |-- description: string (nullable = true)
 |-- id: long (nullable = true)
 |-- link: string (nullable = true)
 |-- slug: string (nullable = true)
 |-- status: string (nullable = true)
 |-- title: string (nullable = true)
 |-- publishedOn: string (nullable = true)



Both `createdOn` and `publishedOn` are stored as strings.

Cast those values to SQL timestamps:

In this case, use a single `select` method to:
0. Cast `dates.publishedOn` to a `timestamp` data type
0. "Flatten" the `dates.publishedOn` column to just `publishedOn`

In [10]:
from pyspark.sql.functions import date_format
display(blogDF.select("title",date_format("dates.publishedOn","yyyy-MM-dd").alias("publishedOn")))

,title,publishedOn
0,MapR Integrates the Complete Apache Spark Stack,2014-04-10
1,Apache Spark 0.9.1 Released,2014-04-10
2,Application Spotlight: Alpine Data Labs,2014-04-01
3,Spark SQL: Manipulating Structured Data Using ...,2014-03-27
4,Apache Spark 0.9.0 Released,2014-02-04
5,Apache Spark In MapReduce (SIMR),2014-01-02
6,Sharethrough Uses Apache Spark Streaming to Op...,2014-03-26
7,Apache Spark: A Delight for Developers,2014-03-21
8,"Databricks announces ""Certified on Apache Spar...",2014-03-19
9,Apache Spark Now a Top-level Apache Project,2014-03-03


Create another DataFrame, `blog2DF` that contains the original columns plus the new `publishedOn` column obtained
from flattening the dates column.

In [11]:
blog2DF = blogDF.withColumn("publishedOn", date_format("dates.publishedOn","yyyy-MM-dd")) 
display(blog2DF)

,authors,categories,content,creator,dates,description,id,link,slug,status,title,publishedOn
0,[Tomer Shiran (VP of Product Management at MapR)],"[Company Blog, Partners]","<div class=""post-meta"">This post is guest auth...",roy,"(2014-04-10, 2014-04-10, UTC)",None,33,https://databricks.com/blog/2014/04/10/mapr-in...,mapr-integrates-spark-stack,publish,MapR Integrates the Complete Apache Spark Stack,2014-04-10
1,[Tathagata Das],"[Apache Spark, Engineering Blog, Machine Learn...",We are happy to announce the availability of <...,tdas,"(2014-04-10, 2014-04-10, UTC)",None,35,https://databricks.com/blog/2014/04/09/spark-0...,spark-0_9_1-released,publish,Apache Spark 0.9.1 Released,2014-04-10
2,[Steven Hillion],"[Company Blog, Partners]","<div class=""post-meta"">This post is guest auth...",roy,"(2014-04-01, 2014-04-01, UTC)",None,37,https://databricks.com/blog/2014/03/31/applica...,application-spotlight-alpine,publish,Application Spotlight: Alpine Data Labs,2014-04-01
3,"[Michael Armbrust, Reynold Xin]","[Apache Spark, Engineering Blog]",Building a unified platform for big data analy...,michael,"(2014-03-27, 2014-03-27, UTC)",None,42,https://databricks.com/blog/2014/03/26/spark-s...,spark-sql-manipulating-structured-data-using-s...,publish,Spark SQL: Manipulating Structured Data Using ...,2014-03-27
4,[Patrick Wendell],"[Apache Spark, Engineering Blog]",Our goal with Apache Spark is very simple: pro...,patrick,"(2014-02-04, 2014-02-04, UTC)",None,58,https://databricks.com/blog/2014/02/03/release...,release-0_9_0,publish,Apache Spark 0.9.0 Released,2014-02-04
5,"[Ali Ghodsi, Ahir Reddy]","[Apache Spark, Ecosystem, Engineering Blog]",Apache Hadoop integration has always been a ke...,ali,"(2014-01-02, 2014-01-02, UTC)",None,65,https://databricks.com/blog/2014/01/01/simr.html,simr,publish,Apache Spark In MapReduce (SIMR),2014-01-02
6,[Russell Cardullo (Data Infrastructure Enginee...,"[Company Blog, Customers]","<div class=""post-meta"">We're very happy to see...",roy,"(2014-03-26, 2014-03-26, UTC)",None,2409,https://databricks.com/blog/2014/03/25/shareth...,sharethrough-and-spark-streaming,publish,Sharethrough Uses Apache Spark Streaming to Op...,2014-03-26
7,"[Jai Ranganathan, Matei Zaharia]","[Apache Spark, Engineering Blog]","<div class=""post-meta"">\n\nThis article was cr...",matei,"(2014-03-21, 2014-03-21, UTC)",None,2410,https://databricks.com/blog/2014/03/20/apache-...,apache-spark-a-delight-for-developers,publish,Apache Spark: A Delight for Developers,2014-03-21
8,[Databricks Press Office],"[Announcements, Company Blog]","<strong>BERKELEY, Calif. – March 18, 2014 –</s...",roy,"(2014-03-19, 2014-03-19, UTC)",None,2411,https://databricks.com/blog/2014/03/18/spark-c...,spark-certification,publish,"Databricks announces ""Certified on Apache Spar...",2014-03-19
9,[Ion Stoica],"[Apache Spark, Engineering Blog]","<div class=""blogContent"">\n\nWe are delighted ...",ion,"(2014-03-03, 2014-03-03, UTC)",None,2412,https://databricks.com/blog/2014/03/02/spark-a...,spark-apache-top-level-project,publish,Apache Spark Now a Top-level Apache Project,2014-03-03


With this temporary view, apply the `printSchema` method to check its schema and confirm the timestamp conversion.

In [12]:
blog2DF.printSchema()

root
 |-- authors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- content: string (nullable = true)
 |-- creator: string (nullable = true)
 |-- dates: struct (nullable = true)
 |    |-- createdOn: string (nullable = true)
 |    |-- publishedOn: string (nullable = true)
 |    |-- tz: string (nullable = true)
 |-- description: string (nullable = true)
 |-- id: long (nullable = true)
 |-- link: string (nullable = true)
 |-- slug: string (nullable = true)
 |-- status: string (nullable = true)
 |-- title: string (nullable = true)
 |-- publishedOn: string (nullable = true)



Since the dates are represented by a `timestamp` data type, we need to convert to a data type that allows `<` and `>`-type comparison operations in order to query for articles within certain date ranges (such as a list of all articles published in 2013). This is accopmplished by using the `to_date` function in Scala or Python.

See the Spark documentation on <a href="https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$" target="_blank">built-in functions</a>, for a long list of date-specific functions.

In [13]:
from pyspark.sql.functions import to_date, year, col
          
resultDF = (blog2DF.select("title", to_date(col("publishedOn"),"MMM dd, yyyy").alias('date'),"link")
            .filter(year(col("publishedOn")) == '2013')
            .orderBy(col("publishedOn")))

display(resultDF)

,title,date,link
0,Databricks and the Apache Spark Platform,None,https://databricks.com/blog/2013/10/27/databri...
1,The Growing Apache Spark Community,None,https://databricks.com/blog/2013/10/27/the-gro...
2,Databricks and Cloudera Partner to Support Apa...,None,https://databricks.com/blog/2013/10/28/databri...
3,Putting Apache Spark to Use: Fast In-Memory Co...,None,https://databricks.com/blog/2013/11/21/putting...
4,Highlights From Spark Summit 2013,None,https://databricks.com/blog/2013/12/18/spark-s...
5,Apache Spark 0.8.1 Released,None,https://databricks.com/blog/2013/12/19/release...


## Array Data

The DataFrame also contains array columns. 

Easily determine the size of each array using the built-in `size(..)` function with array columns.

In [14]:
from pyspark.sql.functions import size
display(blogDF.select(size("authors"),"authors"))

,size(authors),authors
0,1,[Tomer Shiran (VP of Product Management at MapR)]
1,1,[Tathagata Das]
2,1,[Steven Hillion]
3,2,"[Michael Armbrust, Reynold Xin]"
4,1,[Patrick Wendell]
5,2,"[Ali Ghodsi, Ahir Reddy]"
6,2,[Russell Cardullo (Data Infrastructure Enginee...
7,2,"[Jai Ranganathan, Matei Zaharia]"
8,1,[Databricks Press Office]
9,1,[Ion Stoica]


Pull the first element from the array `authors` using an array subscript operator.

For example, in Scala, the 0th element of array `authors` is `authors(0)`
whereas, in Python, the 0th element of `authors` is `authors[0]`.

In [15]:
display(blogDF.select(col("authors")[0].alias("primaryAuthor")))

,primaryAuthor
0,Tomer Shiran (VP of Product Management at MapR)
1,Tathagata Das
2,Steven Hillion
3,Michael Armbrust
4,Patrick Wendell
5,Ali Ghodsi
6,Russell Cardullo (Data Infrastructure Engineer...
7,Jai Ranganathan
8,Databricks Press Office
9,Ion Stoica


### Explode

The `explode` method allows you to split an array column into multiple rows, copying all the other columns into each new row. 

For example, split the column `authors` into the column `author`, with one author per row.

In [16]:
from pyspark.sql.functions import explode
display(blogDF.select("title","authors",explode(col("authors")).alias("author"), "link"))

,title,authors,author,link
0,MapR Integrates the Complete Apache Spark Stack,[Tomer Shiran (VP of Product Management at MapR)],Tomer Shiran (VP of Product Management at MapR),https://databricks.com/blog/2014/04/10/mapr-in...
1,Apache Spark 0.9.1 Released,[Tathagata Das],Tathagata Das,https://databricks.com/blog/2014/04/09/spark-0...
2,Application Spotlight: Alpine Data Labs,[Steven Hillion],Steven Hillion,https://databricks.com/blog/2014/03/31/applica...
3,Spark SQL: Manipulating Structured Data Using ...,"[Michael Armbrust, Reynold Xin]",Michael Armbrust,https://databricks.com/blog/2014/03/26/spark-s...
4,Spark SQL: Manipulating Structured Data Using ...,"[Michael Armbrust, Reynold Xin]",Reynold Xin,https://databricks.com/blog/2014/03/26/spark-s...
5,Apache Spark 0.9.0 Released,[Patrick Wendell],Patrick Wendell,https://databricks.com/blog/2014/02/03/release...
6,Apache Spark In MapReduce (SIMR),"[Ali Ghodsi, Ahir Reddy]",Ali Ghodsi,https://databricks.com/blog/2014/01/01/simr.html
7,Apache Spark In MapReduce (SIMR),"[Ali Ghodsi, Ahir Reddy]",Ahir Reddy,https://databricks.com/blog/2014/01/01/simr.html
8,Sharethrough Uses Apache Spark Streaming to Op...,[Russell Cardullo (Data Infrastructure Enginee...,Russell Cardullo (Data Infrastructure Engineer...,https://databricks.com/blog/2014/03/25/shareth...
9,Sharethrough Uses Apache Spark Streaming to Op...,[Russell Cardullo (Data Infrastructure Enginee...,Michael Ruggiero (Data Infrastructure Engineer...,https://databricks.com/blog/2014/03/25/shareth...


It's more obvious to restrict the output to articles that have multiple authors, and then sort by the title.

In [17]:
blog2DF = (blogDF 
  .select("title","authors",explode(col("authors")).alias("author"), "link") 
  .filter(size(col("authors")) > 1) 
  .orderBy("title")
)

display(blogDF)

,authors,categories,content,creator,dates,description,id,link,slug,status,title
0,[Tomer Shiran (VP of Product Management at MapR)],"[Company Blog, Partners]","<div class=""post-meta"">This post is guest auth...",roy,"(2014-04-10, 2014-04-10, UTC)",None,33,https://databricks.com/blog/2014/04/10/mapr-in...,mapr-integrates-spark-stack,publish,MapR Integrates the Complete Apache Spark Stack
1,[Tathagata Das],"[Apache Spark, Engineering Blog, Machine Learn...",We are happy to announce the availability of <...,tdas,"(2014-04-10, 2014-04-10, UTC)",None,35,https://databricks.com/blog/2014/04/09/spark-0...,spark-0_9_1-released,publish,Apache Spark 0.9.1 Released
2,[Steven Hillion],"[Company Blog, Partners]","<div class=""post-meta"">This post is guest auth...",roy,"(2014-04-01, 2014-04-01, UTC)",None,37,https://databricks.com/blog/2014/03/31/applica...,application-spotlight-alpine,publish,Application Spotlight: Alpine Data Labs
3,"[Michael Armbrust, Reynold Xin]","[Apache Spark, Engineering Blog]",Building a unified platform for big data analy...,michael,"(2014-03-27, 2014-03-27, UTC)",None,42,https://databricks.com/blog/2014/03/26/spark-s...,spark-sql-manipulating-structured-data-using-s...,publish,Spark SQL: Manipulating Structured Data Using ...
4,[Patrick Wendell],"[Apache Spark, Engineering Blog]",Our goal with Apache Spark is very simple: pro...,patrick,"(2014-02-04, 2014-02-04, UTC)",None,58,https://databricks.com/blog/2014/02/03/release...,release-0_9_0,publish,Apache Spark 0.9.0 Released
5,"[Ali Ghodsi, Ahir Reddy]","[Apache Spark, Ecosystem, Engineering Blog]",Apache Hadoop integration has always been a ke...,ali,"(2014-01-02, 2014-01-02, UTC)",None,65,https://databricks.com/blog/2014/01/01/simr.html,simr,publish,Apache Spark In MapReduce (SIMR)
6,[Russell Cardullo (Data Infrastructure Enginee...,"[Company Blog, Customers]","<div class=""post-meta"">We're very happy to see...",roy,"(2014-03-26, 2014-03-26, UTC)",None,2409,https://databricks.com/blog/2014/03/25/shareth...,sharethrough-and-spark-streaming,publish,Sharethrough Uses Apache Spark Streaming to Op...
7,"[Jai Ranganathan, Matei Zaharia]","[Apache Spark, Engineering Blog]","<div class=""post-meta"">\n\nThis article was cr...",matei,"(2014-03-21, 2014-03-21, UTC)",None,2410,https://databricks.com/blog/2014/03/20/apache-...,apache-spark-a-delight-for-developers,publish,Apache Spark: A Delight for Developers
8,[Databricks Press Office],"[Announcements, Company Blog]","<strong>BERKELEY, Calif. – March 18, 2014 –</s...",roy,"(2014-03-19, 2014-03-19, UTC)",None,2411,https://databricks.com/blog/2014/03/18/spark-c...,spark-certification,publish,"Databricks announces ""Certified on Apache Spar..."
9,[Ion Stoica],"[Apache Spark, Engineering Blog]","<div class=""blogContent"">\n\nWe are delighted ...",ion,"(2014-03-03, 2014-03-03, UTC)",None,2412,https://databricks.com/blog/2014/03/02/spark-a...,spark-apache-top-level-project,publish,Apache Spark Now a Top-level Apache Project


## Exercise 1

Identify all the articles written or co-written by Michael Armbrust.

### Step 1

Starting with the `blogDF` DataFrame, create a DataFrame called `articlesByMichaelDF` where:
0. Michael Armbrust is the author.
0. The data set contains the column `title` (it may contain others).
0. It contains only one record per article.

**Hint:** See the Spark documentation on <a href="https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$" target="_blank">built-in functions</a>.  

**Hint:** Include the column `authors` in your view to help you debug your solution.

In [18]:
# TODO
articlesByMichaelDF = (blogDF.select("title", explode(col("authors")).alias("author"))
                       .filter("author == 'Michael Armbrust'")
                       .drop("author"))

In [19]:
# TEST - Run this cell to test your solution.

from pyspark.sql import Row

resultsCount = articlesByMichaelDF.count()
dfTest("DF-L5-articlesByMichael-count", 3, resultsCount)  

results = articlesByMichaelDF.collect()

dfTest("DF-L5-articlesByMichael-0", Row(title=u'Spark SQL: Manipulating Structured Data Using Apache Spark'), results[0])
dfTest("DF-L5-articlesByMichael-1", Row(title=u'Exciting Performance Improvements on the Horizon for Spark SQL'), results[1])
dfTest("DF-L5-articlesByMichael-2", Row(title=u'Spark SQL Data Sources API: Unified Data Access for the Apache Spark Platform'), results[2])

print("Tests passed!")

Tests passed!


### Step 2
Show the list of Michael Armbrust's articles in HTML format.

In [20]:
# TODO
display(articlesByMichaelDF)

,title
0,Spark SQL: Manipulating Structured Data Using ...
1,Exciting Performance Improvements on the Horiz...
2,Spark SQL Data Sources API: Unified Data Acces...


## Exercise 2

Identify the complete set of categories used in the blog articles.

### Step 1

Starting with the `blogDF` DataFrame, create another DataFrame called `uniqueCategoriesDF` where:
0. The data set contains the one column `category` (and no others).
0. This list of categories should be unique.

In [21]:
# TODO
uniqueCategoriesDF = (blogDF.select(explode(col("categories")).alias("category")).distinct().orderBy("category"))

In [22]:
# TEST - Run this cell to test your solution.

resultsCount =  uniqueCategoriesDF.count()

dfTest("DF-L5-uniqueCategories-count", 12, resultsCount)

results = uniqueCategoriesDF.collect()

dfTest("DF-L5-uniqueCategories-0", Row(category=u'Announcements'), results[0])
dfTest("DF-L5-uniqueCategories-1", Row(category=u'Apache Spark'), results[1])
dfTest("DF-L5-uniqueCategories-2", Row(category=u'Company Blog'), results[2])

dfTest("DF-L5-uniqueCategories-9", Row(category=u'Platform'), results[9])
dfTest("DF-L5-uniqueCategories-10", Row(category=u'Product'), results[10])
dfTest("DF-L5-uniqueCategories-11", Row(category=u'Streaming'), results[11])

print("Tests passed!")

Tests passed!


### Step 2
Show the complete list of categories.

In [23]:
# TODO
display(uniqueCategoriesDF)

,category
0,Announcements
1,Apache Spark
2,Company Blog
3,Customers
4,Ecosystem
5,Engineering Blog
6,Events
7,Machine Learning
8,Partners
9,Platform


## Exercise 3

Count how many times each category is referenced in the blog.

### Step 1

Starting with the `blogDF` DataFrame, create another DataFrame called `totalArticlesByCategoryDF` where:
0. The new DataFrame contains two columns, `category` and `total`.
0. The `category` column is a single, distinct category (similar to the last exercise).
0. The `total` column is the total number of articles in that category.
0. Order by `category`.

Because articles can be tagged with multiple categories, the sum of the totals adds up to more than the total number of articles.

In [26]:
# TODO
from pyspark.sql.functions import count
totalArticlesByCategoryDF = (blogDF.select(explode(col("categories")).alias("category"))
                             .groupBy("category")
                             .agg(count("*").alias("total"))
                             .orderBy("category"))

In [27]:
# TEST - Run this cell to test your solution.

results = totalArticlesByCategoryDF.count()

dfTest("DF-L5-articlesByCategory-count", 12, results)

print("Tests passed!")

Tests passed!


In [28]:
# TEST - Run this cell to test your solution.

results = totalArticlesByCategoryDF.collect()

dfTest("DF-L5-articlesByCategory-0", Row(category=u'Announcements', total=72), results[0])
dfTest("DF-L5-articlesByCategory-1", Row(category=u'Apache Spark', total=132), results[1])
dfTest("DF-L5-articlesByCategory-2", Row(category=u'Company Blog', total=224), results[2])

dfTest("DF-L5-articlesByCategory-9", Row(category=u'Platform', total=4), results[9])
dfTest("DF-L5-articlesByCategory-10", Row(category=u'Product', total=83), results[10])
dfTest("DF-L5-articlesByCategory-11", Row(category=u'Streaming', total=21), results[11])

print("Tests passed!")

Tests passed!


### Step 2
Display the totals of each category in html format (should be ordered by `category`).

In [29]:
# TODO
display(totalArticlesByCategoryDF)

,category,total
0,Announcements,72
1,Apache Spark,132
2,Company Blog,224
3,Customers,34
4,Ecosystem,21
5,Engineering Blog,141
6,Events,52
7,Machine Learning,38
8,Partners,50
9,Platform,4


## Summary

* Spark DataFrames allows you to query and manipulate structured and semi-structured data.
* Spark DataFrames built-in functions provide powerful primitives for querying complex schemas.

## Review Questions
**Q:** What is the syntax for accessing nested columns?  
**A:** Use the dot notation:
`select("dates.publishedOn")`

**Q:** What is the syntax for accessing the first element in an array?  
**A:** Use the [subscript] notation: 
`select("col(authors)[0]")`

**Q:** What is the syntax for expanding an array into multiple rows?  
**A:** Use the explode method:  `select(explode(col("authors")).alias("Author"))`

## Additional Topics & Resources

* <a href="http://spark.apache.org/docs/latest/sql-programming-guide.html" target="_blank">Spark SQL, DataFrames and Datasets Guide</a>


&copy; 2019 [Intellinum Analytics, Inc](http://www.intellinum.co). All rights reserved.<br/>